In [44]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pandas as pd

In [45]:
# only run this once ever per session
# try:
#     whole = pd.read_csv("signs.csv")

# except:
#     print("No dataset found")

In [46]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [47]:
cap = cv2.VideoCapture(0)
ls = []
elapsed = 0

with mp_hands.Hands(
    model_complexity = 0,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as hands:
    
    while cap.isOpened():

        time_start = time.time()

        retval, frame = cap.read()

        if not retval:
            print("Camera Error; Exiting")
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

        if results.multi_hand_world_landmarks:
            for hand_landmarks in results.multi_hand_world_landmarks:
                ls.append([[lmk.x, lmk.y, lmk.z] for lmk in hand_landmarks.landmark])


        fps = 1 / (time.time() - time_start)

        frame = cv2.putText(frame, f"FPS: {round(fps)}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        cv2.imshow("Hand Landmarks", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1736802687.555451  171830 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1736802687.556777  172914 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.3.6), renderer: Mesa Intel(R) HD Graphics 630 (KBL GT2)
W0000 00:00:1736802687.572909  172907 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736802687.580776  172910 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
QObject::moveToThread: Current thread (0x2c313e0) is not the object's thread (0x3147190).
Cannot move to target thread (0x2c313e0)

QObject::moveToThread: Current thread (0x2c313e0) is not the object's thread (0x3147190).
Cannot move to target thread (0x2c313e0)

QObject::moveToThread: Current thread (0x2c313e0) is not the object's thread (0x3147190).
Cannot move to target

In [48]:
ls = np.array(ls)

ls

array([[[-1.72470161e-03,  5.15349247e-02,  5.33931106e-02],
        [-2.29277741e-02,  3.31357867e-02,  3.74145098e-02],
        [-3.47465761e-02,  2.34706849e-02,  1.66897662e-02],
        ...,
        [ 1.41151827e-02,  1.85319912e-02, -1.08420476e-02],
        [ 4.59234277e-03,  3.07655316e-02, -8.35583545e-03],
        [ 2.22376734e-03,  4.21976261e-02,  1.32315629e-03]],

       [[-3.74143198e-02,  1.60439014e-02,  5.82642667e-02],
        [-1.94105953e-02,  3.08222435e-02,  3.33188437e-02],
        [-5.21032419e-03,  4.67885099e-02,  1.57332104e-02],
        ...,
        [-1.70505475e-02, -2.94028036e-02,  3.49053560e-04],
        [-1.99639294e-02, -2.60567646e-02, -1.38034970e-02],
        [-2.49339491e-02, -1.51843708e-02, -1.84346177e-02]],

       [[-1.75524764e-02,  5.38650490e-02,  4.93588373e-02],
        [-1.70646068e-02,  3.38949300e-02,  3.67310233e-02],
        [-1.31905358e-02,  2.28230637e-02,  1.92643777e-02],
        ...,
        [ 7.36728916e-03, -8.61707423e-03,

In [49]:
reshaped_ls = ls.copy()
reshaped_ls = reshaped_ls.reshape(-1, 63)
reshaped_ls.shape

(587, 63)

In [50]:
df_columns = []
for i in range(21):
    df_columns.append(f"Landmark {i} x")
    df_columns.append(f"Landmark {i} y")
    df_columns.append(f"Landmark {i} z")

In [51]:
# create dataframe where rows are frames and columns are coordinates of all 21 landmarks (x, y have distinct columns)
df = pd.DataFrame(reshaped_ls, columns=df_columns)
df.shape

(587, 63)

In [52]:
# add label for handpose (DO NOT USE EMOJIS LOL)
df["Label"] = "not good"

In [53]:
try:
    whole = pd.concat([whole, df], axis=0)
except:
    whole = df

whole.shape

(1712, 64)

In [54]:
whole["Label"].value_counts()

Label
okay        732
not good    587
good        393
Name: count, dtype: int64

In [55]:
whole[whole["Label"] == "good"].head()

,Landmark 0 x,Landmark 0 y,Landmark 0 z,Landmark 1 x,Landmark 1 y,Landmark 1 z,Landmark 2 x,Landmark 2 y,Landmark 2 z,Landmark 3 x,...,Landmark 18 x,Landmark 18 y,Landmark 18 z,Landmark 19 x,Landmark 19 y,Landmark 19 z,Landmark 20 x,Landmark 20 y,Landmark 20 z,Label
0,-0.037530,0.025283,0.071331,-0.023538,-0.002247,0.051992,-0.021551,-0.026065,0.046599,-0.020290,...,0.003695,0.023843,-0.004019,-0.004343,0.022965,-0.001415,-0.014656,0.025710,0.000322,good
1,-0.043660,0.023239,0.063787,-0.021679,-0.005747,0.051859,-0.015757,-0.025957,0.040206,-0.013190,...,0.014027,0.024153,0.017877,0.009851,0.025378,0.035494,0.002660,0.032693,0.043769,good
2,-0.051530,0.028312,0.058247,-0.027081,0.000099,0.048187,-0.019565,-0.020041,0.039265,-0.013668,...,0.013781,0.025802,0.018954,0.008727,0.027022,0.041238,0.001672,0.033809,0.051694,good
3,-0.052828,0.031573,0.058133,-0.028352,0.002001,0.050642,-0.021189,-0.018299,0.041453,-0.012829,...,0.015558,0.027155,0.020076,0.009841,0.029779,0.042574,0.002923,0.035615,0.053516,good
4,-0.051685,0.031216,0.061570,-0.027190,0.002354,0.052111,-0.019695,-0.018768,0.042904,-0.011356,...,0.014085,0.028600,0.019067,0.008531,0.031537,0.040469,0.000791,0.036464,0.049544,good


In [56]:
# just in case of bad data
def delete_label(df, label):
    return df[df["Label"] != label]

# whole = delete_label(whole, "okay")
# whole["Label"].value_counts()

In [58]:
whole.to_csv("new_signs_en.csv", index=False)